# Описание задачи
У нас есть таблица с 2 листами. Первый лист - это исходные данные, а второй - то, как должна выглядеть таблица после нашей обработки. Приступим к работе.

In [1]:
#импортируем необходимые библиотеки
import pandas as pd
from io import BytesIO
import requests
import numpy as np
from nltk.stem import SnowballStemmer 
russian_stemmer = SnowballStemmer('russian')  

In [2]:
#встраиваем файл в проект
spreadsheet_id = '1XHwnmuZpaAlrCpClVRLfM4p_WwGqmeHiIPyToLj5qDo'
file_name = 'https://docs.google.com/spreadsheets/d/{}/export?format=csv'.format(spreadsheet_id)
r = requests.get(file_name)
#присваиваем переменной значение датасета
logs = pd.read_csv(BytesIO(r.content))
logs

,№,Диагноз,Код \nуслуги,Название услуги,Unnamed: 4,Unnamed: 5,Цена по прейскуранту,Цена со скидкой,Кол-\nво,Сумма со скидкой
0,Иванов Иван Иванович Полис...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,13.07.2022,NaN,Врач Горелов,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,H52.0,1.2.4,"Прием (осмотр, консультация) врача-ревматоло...",NaN,NaN,1112.0,1112.0,1.0,1112.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,14.07.2022,NaN,Врач Аксенов,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,J30.0 J34.2 K21 H91.1,1.31.1,"Прием (осмотр, консультация) врача-терапевта...",NaN,NaN,1112.0,1112.0,1.0,1112.0
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Так выглядит таблица сейчас, нам же нужно ее преобразовать так, чтобы она выглядела, как таблица, представленная ниже:

In [3]:
#встраиваем файл в проект
spreadsheet_id = '1zHUubbRrySrvD-S5EfEXRtYMuGZeLFFhDqTBy3VmDOY'
file_name = 'https://docs.google.com/spreadsheets/d/{}/export?format=csv'.format(spreadsheet_id)
r = requests.get(file_name)
#присваиваем переменной значение датасета
result = pd.read_csv(BytesIO(r.content))
result

,ФИО,Номер полиса,Дата оказания услуги,Код услуги,Диагноз,Наименование услуги,Кол-во,Цена по прейскуранту,Сумма со скидкой
0,Иванов Иван Иванович,23-555555/10,13.07.2022,1.2.4,H52.0,"Прием (осмотр, консультация) врача-ревматоло...",1,1112,1112
1,Иванов Иван Иванович,23-555555/10,14.07.2022,1.31.1,J30.0 J34.2 K21 H91.1,"Прием (осмотр, консультация) врача-терапевта...",1,1112,1112
2,Петров Петр Петрович,23-555555/27,19.07.2022,1.8.1,M54.2 M54.6 M54.1,"Прием (осмотр, консультация) врача-пульмонол...",1,1073,1073


Нам придется собирать таблицу по частям, поскольку конструция исходной таблицы сложна для того, чтобы полностью ее преобразовать. Мы будем работать отдельно со столбцами и ивлекать из них необходимую информацию.

In [4]:
#начнем с первого столбца
number = logs['№'].unique()
#удалим значений NaN
number = [x.strip() for x in number if not pd.isna(x)]
#разделим каждую строку на две части, используя разделитель "Полис №"
split_data = [item.split('Полис №') for item in number]
split_data.append(split_data[1])
split_data[1]=split_data[0]
split_data

[['Иванов Иван Иванович                     ', ' 23-555555/10'],
 ['Иванов Иван Иванович                     ', ' 23-555555/10'],
 ['Петров Петр Петрович        ', ' 23-555555/27']]

Поскольку Иванов Иван Иванович имеет два похода к врачу мы его дважды записали

In [5]:
#создадим DataFrame из разделенных данных
df = pd.DataFrame(split_data, columns=['ФИО', 'Номер полиса'])
df['ФИО'] =df['ФИО'].str.lstrip()
df['ФИО'] =df['ФИО'].str.rstrip()
#отобразим полученную таблицу
df

,ФИО,Номер полиса
0,Иванов Иван Иванович,23-555555/10
1,Иванов Иван Иванович,23-555555/10
2,Петров Петр Петрович,23-555555/27


Мы извлекли первую часть данных с информацией о клиентах.

In [6]:
dia = logs['Диагноз'].unique()
# Удаление значений NaN
dia  = [x.strip() for x in dia  if not pd.isna(x)]
dia= [[dia[i], dia[i+1]] for i in range(0, len(dia), 2)]
# Создайте DataFrame из разделенных данных
df1 = pd.DataFrame(dia, columns=['Дата оказания услуги', 'Диагноз'])
df1

,Дата оказания услуги,Диагноз
0,13.07.2022,H52.0
1,14.07.2022,J30.0 J34.2 K21 H91.1
2,19.07.2022,M54.2 M54.6 M54.1


In [7]:
#объединим таблицы
table = df.join(df1)
table

,ФИО,Номер полиса,Дата оказания услуги,Диагноз
0,Иванов Иван Иванович,23-555555/10,13.07.2022,H52.0
1,Иванов Иван Иванович,23-555555/10,14.07.2022,J30.0 J34.2 K21 H91.1
2,Петров Петр Петрович,23-555555/27,19.07.2022,M54.2 M54.6 M54.1


In [8]:
#извлечем информацию о коде услуги
kod = logs['Код \nуслуги'].unique() 
kod= [x.strip() for x in kod  if not pd.isna(x)]
#добавим столбец в нашу таблицу
table['Код услуги'] = kod
table

,ФИО,Номер полиса,Дата оказания услуги,Диагноз,Код услуги
0,Иванов Иван Иванович,23-555555/10,13.07.2022,H52.0,1.2.4
1,Иванов Иван Иванович,23-555555/10,14.07.2022,J30.0 J34.2 K21 H91.1,1.31.1
2,Петров Петр Петрович,23-555555/27,19.07.2022,M54.2 M54.6 M54.1,1.8.1


In [10]:
#работаем со столбцом с названиями услуг
name = logs['Название услуги'].unique()
# Удаление значений NaN
name = [x.strip() for x in name if not pd.isna(x)]
name= [name[i+1] for i in range(0, len(name), 2)]
#добавим столбец в нашу таблицу
table['Наименование услуги'] = name
table

,ФИО,Номер полиса,Дата оказания услуги,Диагноз,Код услуги,Наименование услуги
0,Иванов Иван Иванович,23-555555/10,13.07.2022,H52.0,1.2.4,"Прием (осмотр, консультация) врача-ревматолога..."
1,Иванов Иван Иванович,23-555555/10,14.07.2022,J30.0 J34.2 K21 H91.1,1.31.1,"Прием (осмотр, консультация) врача-терапевта п..."
2,Петров Петр Петрович,23-555555/27,19.07.2022,M54.2 M54.6 M54.1,1.8.1,"Прием (осмотр, консультация) врача-пульмонолог..."


In [11]:
#теперь работаем со столбцом количество услуг
counts = logs['Кол-\nво']
# Удаление значений NaN
counts = [x for x in counts if not pd.isna(x)]
table['Кол-во'] = counts
table

,ФИО,Номер полиса,Дата оказания услуги,Диагноз,Код услуги,Наименование услуги,Кол-во
0,Иванов Иван Иванович,23-555555/10,13.07.2022,H52.0,1.2.4,"Прием (осмотр, консультация) врача-ревматолога...",1.0
1,Иванов Иван Иванович,23-555555/10,14.07.2022,J30.0 J34.2 K21 H91.1,1.31.1,"Прием (осмотр, консультация) врача-терапевта п...",1.0
2,Петров Петр Петрович,23-555555/27,19.07.2022,M54.2 M54.6 M54.1,1.8.1,"Прием (осмотр, консультация) врача-пульмонолог...",1.0


In [12]:
#теперь работаем со столбцом цены
price = logs['Цена по прейскуранту']
# Удаление значений NaN
price = [x for x in price if not pd.isna(x)]
#добавим столбец в нашу таблицу
table['Цена по прейскуранту'] = price
table

,ФИО,Номер полиса,Дата оказания услуги,Диагноз,Код услуги,Наименование услуги,Кол-во,Цена по прейскуранту
0,Иванов Иван Иванович,23-555555/10,13.07.2022,H52.0,1.2.4,"Прием (осмотр, консультация) врача-ревматолога...",1.0,1112.0
1,Иванов Иван Иванович,23-555555/10,14.07.2022,J30.0 J34.2 K21 H91.1,1.31.1,"Прием (осмотр, консультация) врача-терапевта п...",1.0,1112.0
2,Петров Петр Петрович,23-555555/27,19.07.2022,M54.2 M54.6 M54.1,1.8.1,"Прием (осмотр, консультация) врача-пульмонолог...",1.0,1073.0


In [13]:
#теперь работаем со столбцом цены
discount = logs['Сумма со скидкой']
# Удаление значений NaN
discount = [x for x in discount if not pd.isna(x)]
#добавим столбец в нашу таблицу
table['Сумма со скидкой'] = discount
table

,ФИО,Номер полиса,Дата оказания услуги,Диагноз,Код услуги,Наименование услуги,Кол-во,Цена по прейскуранту,Сумма со скидкой
0,Иванов Иван Иванович,23-555555/10,13.07.2022,H52.0,1.2.4,"Прием (осмотр, консультация) врача-ревматолога...",1.0,1112.0,1112.0
1,Иванов Иван Иванович,23-555555/10,14.07.2022,J30.0 J34.2 K21 H91.1,1.31.1,"Прием (осмотр, консультация) врача-терапевта п...",1.0,1112.0,1112.0
2,Петров Петр Петрович,23-555555/27,19.07.2022,M54.2 M54.6 M54.1,1.8.1,"Прием (осмотр, консультация) врача-пульмонолог...",1.0,1073.0,1073.0


Теперь в нашей таблице есть вся нужная информация о том, что дожно быть в итоговой. Посмотрим на внешний вид итоговой таблицы и сравним ее с нашей

In [14]:
result

,ФИО,Номер полиса,Дата оказания услуги,Код услуги,Диагноз,Наименование услуги,Кол-во,Цена по прейскуранту,Сумма со скидкой
0,Иванов Иван Иванович,23-555555/10,13.07.2022,1.2.4,H52.0,"Прием (осмотр, консультация) врача-ревматоло...",1,1112,1112
1,Иванов Иван Иванович,23-555555/10,14.07.2022,1.31.1,J30.0 J34.2 K21 H91.1,"Прием (осмотр, консультация) врача-терапевта...",1,1112,1112
2,Петров Петр Петрович,23-555555/27,19.07.2022,1.8.1,M54.2 M54.6 M54.1,"Прием (осмотр, консультация) врача-пульмонол...",1,1073,1073


Мы видим, что нам нужно исправить две вещи:
* Порядок, в котором расположены столбцы
* Тип численных значений - вместо float нужен int

In [15]:
#меняем тип значения
table[['Кол-во', 'Цена по прейскуранту', 'Сумма со скидкой']] = table[['Кол-во', 'Цена по прейскуранту', 'Сумма со скидкой']].astype('int')

In [16]:
#меняем порядок
table = table[['ФИО','Номер полиса', 'Дата оказания услуги', 'Код услуги', 'Диагноз', 'Наименование услуги', 'Кол-во','Цена по прейскуранту', 'Сумма со скидкой']]

Теперь можем вывести таблицу

In [17]:
#выводим нашу таблицу для сравнения
table

,ФИО,Номер полиса,Дата оказания услуги,Код услуги,Диагноз,Наименование услуги,Кол-во,Цена по прейскуранту,Сумма со скидкой
0,Иванов Иван Иванович,23-555555/10,13.07.2022,1.2.4,H52.0,"Прием (осмотр, консультация) врача-ревматолога...",1,1112,1112
1,Иванов Иван Иванович,23-555555/10,14.07.2022,1.31.1,J30.0 J34.2 K21 H91.1,"Прием (осмотр, консультация) врача-терапевта п...",1,1112,1112
2,Петров Петр Петрович,23-555555/27,19.07.2022,1.8.1,M54.2 M54.6 M54.1,"Прием (осмотр, консультация) врача-пульмонолог...",1,1073,1073


In [18]:
#выводим итоговую таблицу
result

,ФИО,Номер полиса,Дата оказания услуги,Код услуги,Диагноз,Наименование услуги,Кол-во,Цена по прейскуранту,Сумма со скидкой
0,Иванов Иван Иванович,23-555555/10,13.07.2022,1.2.4,H52.0,"Прием (осмотр, консультация) врача-ревматоло...",1,1112,1112
1,Иванов Иван Иванович,23-555555/10,14.07.2022,1.31.1,J30.0 J34.2 K21 H91.1,"Прием (осмотр, консультация) врача-терапевта...",1,1112,1112
2,Петров Петр Петрович,23-555555/27,19.07.2022,1.8.1,M54.2 M54.6 M54.1,"Прием (осмотр, консультация) врача-пульмонол...",1,1073,1073


Вывод: мы смогли взять информацию из первого листа нашей таблицы и скомпоновать ее во второй лист таблицы.